In [ ]:
# Code based on:
# *** Forest fire ***
#
# Copyright 2008-2012 Hiroki Sayama
# sayama@binghamton.edu
#
# Ported to Python 3 and added parameter setters
# by Przemyslaw Szufel & Bogumil Kaminski
# {pszufe, bkamins}@sgh.waw.pl
# 
# Modified for ISZ course purposes by Grzegorz Szeremeta and Dariusz Szymczyk
#
import matplotlib
#matplotlib.use('qt4agg')

import pylab as PL
import random as RD
import scipy as SP

RD.seed()

width = 100
height = 100
bad_guys = 50
good_guys = 200

good = 1
bad = -1
empty = 0

def is_bad(val):
    return 0>val>-1

def is_good(val):
    return 0<val<1

def init():
    global time, config, nextConfig
    bad_cnt = bad_guys
    good_cnt = good_guys
    time = 0

    config = SP.zeros([height, width])
    for x in range(width):
        for y in range(height):
            if 0<RD.random() < 0.1 and bad_cnt>0:
                state = bad
                bad_cnt-=1
            elif 0.1<RD.random()<0.5 and good_cnt>0:
                state = good
                good_cnt-=1
            else:
                state = empty
            config[y, x] = state
    while bad_cnt>0:
        x = abs(RD.random()*(width-1))
        y = abs(RD.random()*(height-1))
        if config[y,x] == empty:
            config[y,x]=bad
            bad_cnt-=1
    while good_cnt>0:
        x = abs(RD.random()*(width-1))
        y = abs(RD.random()*(height-1))
        if config[y,x] == empty:
            config[y,x]=good
            good_cnt-=1
    nextConfig = SP.zeros([height, width])

def draw():
    PL.cla()
    PL.pcolor(config, vmin = -1, vmax = 1, cmap = PL.cm.binary)
    PL.axis('image')
    PL.title('t = ' + str(time))

def find_empty(x,y):
    global config
    for dx in range(-1, 2):
        new_x = (x+dx)%width
        for dy in range(-1, 2):
            new_y=(y+dy)%height
            if config[new_y,new_x] == empty:
                return new_y,new_x
    return -1,-1
def create_offspring(x,y):
    global config,nextConfig, good_guys, bad_guys
    new_y,new_x = find_empty(x,y)
    if RD.random()<0.5:
        nextConfig[new_y,new_x] = bad
    else:
        nextConfig[new_y,new_x] = good
        
        
def check_surrounding_good(y,x):
    global config
    for dx in range(-1, 2):
        for dy in range(-1, 2):
            if config[(y+dy)%height, (x+dx)%width] == good:
                create_offspring(x,y)
                config[y,x]-=.1
                return 

def check_surrounding_bad(y,x):
    global config, good_guys
    for dx in range(-1, 2):
        nb_x = (x+dx)%width
        for dy in range(-1, 2):
            nb_y = (y+dy)%height
            if config[nb_y,nb_x] == good:
                nextConfig[nb_y,nb_x] = empty
                good_guys-=1
                config[y,x] -=.1
                return 
                            
    
def step():
    global time, config, nextConfig,good_guys,bad_guys

    time += 1

    height, width = config.shape

    for x in range(width):
        for y in range(height):
            state = config[y, x]
            if state == good:
                check_surrounding_good(y,x)
                new_y,new_x = find_empty(x,y)
                if config[y,x]>0.2:
                    nextConfig[new_y,new_x] = (config[y,x] -0.1)
                else:
                    good_guys-=1
                    nextConfig[new_y,new_x] = empty
            elif state == bad:
                check_surrounding_bad(y,x)
                new_y,new_x = find_empty(x,y)
                if config[y,x]<-0.2:
                    nextConfig[new_y,new_x] = (config[y,x] +0.1)
                else:
                    bad_guys-=1
                    nextConfig[new_y,new_x] = empty
                

    config, nextConfig = nextConfig, config

import pycxsimulator
pycxsimulator.GUI().start(func=[init,draw,step])

